In [24]:
import pandas as pd
import numpy as np
import string
import lxml.etree
import urllib
import urllib.request
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from ast import literal_eval
import matplotlib.pyplot as plt
from textstat.textstat import textstat
from gensim.corpora import wikicorpus
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pandas as pd
import pymongo
from selenium.common.exceptions import NoSuchElementException
from typing import List, Tuple, Dict, Any, Generator, Iterable
import nltk
from collections import defaultdict
import pymongo
from pymongo import MongoClient
import pickle
%matplotlib inline
# Make it pretty
plt.style.use('ggplot')

In [25]:
def browse_to_category(browser, category):
    """ Use browser to get Wikipedia Category page."""
    url = 'https://en.wikipedia.org/wiki/Category:' + category
    browser.get(url)

def get_category_title(browser):
    """ Use css tag 'h1.firstHeading' to find the main category."""
    headline = browser.find_elements_by_css_selector('h1.firstHeading')
    title = [text.text for text in headline]
    return title[0].partition('Category:')[2]

def get_sub_categories(browser):
    """ Use css tag 'a.CategoryTreeLabel' to find the sub-categories."""
    subs = browser.find_elements_by_css_selector('a.CategoryTreeLabel')
    return [category.text for category in subs]

def click_to_sub_category_page(browser, page):
    """ Click a sub_category and be taken to its pages"""
    browser.find_element_by_link_text(page).click()

def filter_pages(pages):
    for page in pages:
        if len(page) < 2:
            pages.remove(page)
    return pages

def get_wiki_xml(title):
    title = title
    params = { "format":"xml", "action":"query", "prop":"revisions", "rvprop":"timestamp|user|comment|content" }
    params["titles"] = "API|%s" % urllib.parse.quote(title.encode("utf8"))
    qs = "&".join("%s=%s" % (k, v)  for k, v in params.items())
    url = "http://en.wikipedia.org/w/api.php?%s" % qs
    tree = lxml.etree.parse(urllib.request.urlopen(url))
    revs = tree.xpath('//rev')
    return (revs[-1].text)

def clean_wiki_markup(raw_article):
    semi_cleaned_article = wikicorpus.filter_wiki(raw_article)
    cleaned_article = semi_cleaned_article.replace("\n", "").replace("\'", "").replace("()", "").replace("=", "").replace("|alt","").replace("\xa0","")
    return cleaned_article
def find_num_categories(raw_article):
    return raw_article.count("[[Category:")
def find_num_images(raw_article):
    return raw_article.count("[[Image:")
def find_num_ISBN(raw_article):
    return raw_article.count("ISBN")
def find_num_references(raw_article):
    return raw_article.count("</ref>")
def find_article_length(cleaned_article):
    return len(cleaned_article)
def find_num_difficult_words(cleaned_article):
    return textstat.difficult_words(cleaned_article)
def find_dale_chall_readability_score(cleaned_article):
    return textstat.dale_chall_readability_score(cleaned_article)
def find_automated_readability_index(cleaned_article):
    return textstat.automated_readability_index(cleaned_article)
def find_linsear_write_formula(cleaned_article):
    return textstat.linsear_write_formula(cleaned_article)
def find_gunning_fog_index(cleaned_article):
    return textstat.gunning_fog(cleaned_article)
def find_syllable_count(cleaned_article):
    return textstat.syllable_count(cleaned_article)
def find_lexicon_count(cleaned_article):
    return textstat.lexicon_count(cleaned_article, removepunct=True)
def find_sentence_count(cleaned_article):
    return textstat.sentence_count(cleaned_article)
def find_smog_index(cleaned_article):
    return textstat.smog_index(cleaned_article)
def find_num_web_citations(raw_article):
    return raw_article.count("{{cite web")
def find_num_book_citations(raw_article):
    return raw_article.count("{{cite book")
def find_num_news_citations(raw_article):
    return raw_article.count("{{cite news")
def find_num_quotes(raw_article):
    return raw_article.count("quote=")
def find_num_h3_headers(raw_article):
    return raw_article.count("\n===")
def find_num_internal_links(raw_article):
    return (raw_article.count("[[") // 2)
def find_num_h2_headers(raw_article):
    return (raw_article.count("\n==") - find_num_h3_headers(raw_article))
def find_num_note_tags(raw_article):
    return raw_article.count("{{note")
def find_num_bullet_points(raw_article):
    return (raw_article.count("*"))
def find_num_underlines(raw_article):
    return (raw_article.count("<u>"))
def find_num_journal_citations(raw_article):
    return (raw_article.count("{{cite journal"))
def find_num_about_links(raw_article):
    return (raw_article.count("{{About"))
def find_num_wikitables(raw_article):
    return (raw_article.count('class="wikitable'))
def find_num_footnotes(raw_article):
    return raw_article.count("{{")
def find_infobox(raw_article):
    return int('{{Infobox' in raw_article)

# Process Start-to-finish:

## 1) Go to wikipedia category page

In [26]:
category = 'machine_learning'
browser = Chrome()

In [27]:
browse_to_category(browser, category)

## 2) Grab Overarching Category title

In [28]:
Overarching_title = get_category_title(browser)

In [29]:
Overarching_title

'Machine learning'

## 3) Grab Subcategories

In [30]:
subs = get_sub_categories(browser)

In [31]:
subs

['Applied machine learning',
 'Artificial neural networks',
 'Bayesian networks',
 'Classification algorithms',
 'Cluster analysis',
 'Computational learning theory',
 'Artificial intelligence conferences',
 'Signal processing conferences',
 'Data mining and machine learning software',
 'Datasets in machine learning',
 'Dimension reduction',
 'Ensemble learning',
 'Evolutionary algorithms',
 'Genetic programming',
 'Inductive logic programming',
 'Kernel methods for machine learning',
 'Latent variable models',
 'Learning in computer vision',
 'Log-linear models',
 'Loss functions',
 'Machine learning algorithms',
 'Machine learning portal',
 'Machine learning task',
 'Markov models',
 'Machine learning researchers',
 'Semisupervised learning',
 'Statistical natural language processing',
 'Structured prediction',
 'Supervised learning',
 'Support vector machines',
 'Unsupervised learning']

## 4) Grab Subcategory information and store in MongoDB

Create MongoDB

In [33]:
client = MongoClient('mongodb://localhost:27017/')
new_db = client['new_db']
collection = new_db['new_db']

Iterate over sub-categories and add their subsquent pages in MongoDB

In [34]:
for sub_cat in subs:
    browse_to_category(browser, category)
    cick_to_sub_category_page(sub_cat)
    title = get_category_title(browser)
    print(title)
    try:
        w = browser.find_element_by_class_name('mw-category')
        pages = filter_pages(w.text.split('\n'))
        for page in pages:
            post = {'category': title,
                'page': page,
                'text' : get_wiki_xml(page)}
            posts = new_db.posts
            post = posts.insert_one(post)
    except NoSuchElementException:
        w = browser.find_elements_by_class_name('mw-content-ltr')
        pages = filter_pages(w[0].text.split('\n'))[2:]
        for page in pages:
            post = {'category': title,
                'page': page,
                'text' : get_wiki_xml(page)}
            posts = new_db.posts
            post = posts.insert_one(post)

Applied machine learning
Artificial neural networks
Bayesian networks
Classification algorithms
Cluster analysis
Computational learning theory
Artificial intelligence conferences
Signal processing conferences
Data mining and machine learning software
Datasets in machine learning
Dimension reduction
Ensemble learning
Evolutionary algorithms
Genetic programming
Inductive logic programming
Kernel methods for machine learning
Latent variable models
Learning in computer vision
Log-linear models
Loss functions
Machine learning algorithms
Machine learning portal
Machine learning task
Markov models
Machine learning researchers
Semisupervised learning
Statistical natural language processing
Structured prediction
Supervised learning
Support vector machines
Unsupervised learning


## 5) Convert MongoDB to pandas DF and clean/transform data

In [35]:
data = pd.DataFrame(list(posts.find()))

In [37]:
data.drop(columns='_id',inplace=True)

In [39]:
data = data[data['text'] != ""]
data = data[data['text'].str.contains("#redirect") == False]
data = data[data['text'].str.contains("may refer to:\n\n*") == False]
data = data[data['text'].str.contains("can refer to:\n") == False]
data = data[data['text'].str.contains("could refer to:\n") == False]
data = data[data['text'].str.contains("#REDIRECT") == False]
data = data[data['text'].str.contains("== Matches ==\n:") == False]
data = data[data['text'].str.contains("{{underconstruction") == False]

In [40]:
data['cleaned_text'] = data['text'].apply(clean_wiki_markup)
data['num_web_citations'] = data['text'].apply(find_num_web_citations)
data['num_book_citations'] = data['text'].apply(find_num_book_citations)
data['num_news_citations'] = data['text'].apply(find_num_news_citations)
data['num_quotes'] = data['text'].apply(find_num_quotes)
data['num_h3_headers'] = data['text'].apply(find_num_h3_headers)
data['num_internal_links'] = data['text'].apply(find_num_internal_links)
data['num_h2_headers'] = data['text'].apply(find_num_h2_headers)
data['has_infobox'] = data['text'].str.contains('{{Infobox').astype(int)
data['num_categories'] = data['text'].apply(find_num_categories)
data['num_images'] = data['text'].apply(find_num_images)
data['num_ISBN'] = data['text'].apply(find_num_ISBN)
data['num_references'] = data['text'].apply(find_num_references)
data['article_length'] = data['text'].apply(find_article_length)
data['num_difficult_words'] = data['cleaned_text'].apply(find_num_difficult_words)
data['dale_chall_readability_score'] = data['cleaned_text'].apply(find_dale_chall_readability_score)
data['readability_index'] = data['cleaned_text'].apply(find_automated_readability_index)
data['linsear_write_formula'] = data['cleaned_text'].apply(find_linsear_write_formula)
data['gunning_fog_index'] = data['cleaned_text'].apply(find_gunning_fog_index)
data['smog_index'] = data['cleaned_text'].apply(find_smog_index)
data['syllable_count'] = data['cleaned_text'].apply(find_syllable_count)
data['lexicon_count'] = data['cleaned_text'].apply(find_lexicon_count)
data['sentence_count'] = data['cleaned_text'].apply(find_sentence_count)
data['num_footnotes'] = data['text'].apply(find_num_footnotes)
data['num_note_tags'] = data['text'].apply(find_num_note_tags)
data['num_underlines'] = data['text'].apply(find_num_underlines)
data['num_journal_citations'] = data['text'].apply(find_num_journal_citations)
data['num_about_links'] = data['text'].apply(find_num_about_links)
data['num_wikitables'] = data['text'].apply(find_num_wikitables)

Error(DCRS): Word Count is zero cannot divide
Error(ARI) : Sentence count is zero, cannot divide
Error(GF): Word Count is Zero, cannot divide


In [41]:
data.dropna(inplace=True)

## 6) Pickle DataFrame

In [ ]:
data.to_pickle('Machine_Learning_Category_Wikipedia.pkl')

## 7) Split data into corrext input (X)

In [42]:
X = data.loc[:, ['has_infobox','num_categories','num_images','num_ISBN','num_references','article_length',
                'num_difficult_words','dale_chall_readability_score','readability_index','linsear_write_formula',
                'gunning_fog_index', 'num_web_citations','num_book_citations','num_news_citations',
                'num_quotes','num_h3_headers','num_internal_links', 'num_h2_headers', 'syllable_count',
                'lexicon_count', 'sentence_count','num_footnotes', 'num_note_tags', 'num_underlines', 'num_journal_citations',
                'num_about_links', 'num_wikitables', 'smog_index']].values

In [43]:
rf_model = pickle.load(open('../src/random_forest_model.sav', 'rb'))

/Users/austin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [44]:
predictions = rf_model.predict(X)

In [45]:
data['Predicted_Quality'] = predictions

In [47]:
Cat_page_df = data.loc[:, ['category', 'page', 'Predicted_Quality']]

In [49]:
Cat_groups = Cat_page_df.groupby(by='category').mean()

In [ ]:
ML_page_predictions = data

In [ ]:
ML_page_predictions.to_pickle('ML_page_predictions.pkl')

In [11]:
def get_wiki_category_quality(category):
    category = category
    browser = Chrome()
    browse_to_category(browser, category)
    client = MongoClient('mongodb://localhost:27017/')
    category_db = client['category_db']
    collection = dogs_db['category_db']
    subs = get_sub_categories(browser)
    for sub_cat in subs:
        browse_to_category(browser, category)
        time.sleep(2)
        click_to_sub_category_page(browser, sub_cat)
        title = get_category_title(browser)
        try:
            w = browser.find_element_by_class_name('mw-category')
            pages = filter_pages(w.text.split('\n'))
            for page in pages:
                post = {'category': title,
                    'page': page,
                    'text' : get_wiki_xml(page)}
                posts = dogs_db.posts
                post = posts.insert_one(post)
        except NoSuchElementException:
            w = browser.find_elements_by_class_name('mw-content-ltr')
            pages = filter_pages(w[0].text.split('\n'))[2:]
            for page in pages:
                post = {'category': title,
                    'page': page,
                    'text' : get_wiki_xml(page)}
                posts = dogs_db.posts
                post = posts.insert_one(post)
    data = pd.DataFrame(list(posts.find()))
    data.drop(columns='_id',inplace=True)
    data = data[data['text'] != ""]
    data = data[data['text'].str.contains("#redirect") == False]
    data = data[data['text'].str.contains("may refer to:\n\n*") == False]
    data = data[data['text'].str.contains("can refer to:\n") == False]
    data = data[data['text'].str.contains("could refer to:\n") == False]
    data = data[data['text'].str.contains("#REDIRECT") == False]
    data = data[data['text'].str.contains("== Matches ==\n:") == False]
    data = data[data['text'].str.contains("{{underconstruction") == False]
    data['cleaned_text'] = data['text'].apply(clean_wiki_markup)
    data['num_web_citations'] = data['text'].apply(find_num_web_citations)
    data['num_book_citations'] = data['text'].apply(find_num_book_citations)
    data['num_news_citations'] = data['text'].apply(find_num_news_citations)
    data['num_quotes'] = data['text'].apply(find_num_quotes)
    data['num_h3_headers'] = data['text'].apply(find_num_h3_headers)
    data['num_internal_links'] = data['text'].apply(find_num_internal_links)
    data['num_h2_headers'] = data['text'].apply(find_num_h2_headers)
    data['has_infobox'] = data['text'].str.contains('{{Infobox').astype(int)
    data['num_categories'] = data['text'].apply(find_num_categories)
    data['num_images'] = data['text'].apply(find_num_images)
    data['num_ISBN'] = data['text'].apply(find_num_ISBN)
    data['num_references'] = data['text'].apply(find_num_references)
    data['article_length'] = data['text'].apply(find_article_length)
    data['num_difficult_words'] = data['cleaned_text'].apply(find_num_difficult_words)
    data['dale_chall_readability_score'] = data['cleaned_text'].apply(find_dale_chall_readability_score)
    data['readability_index'] = data['cleaned_text'].apply(find_automated_readability_index)
    data['linsear_write_formula'] = data['cleaned_text'].apply(find_linsear_write_formula)
    data['gunning_fog_index'] = data['cleaned_text'].apply(find_gunning_fog_index)
    data['smog_index'] = data['cleaned_text'].apply(find_smog_index)
    data['syllable_count'] = data['cleaned_text'].apply(find_syllable_count)
    data['lexicon_count'] = data['cleaned_text'].apply(find_lexicon_count)
    data['sentence_count'] = data['cleaned_text'].apply(find_sentence_count)
    data['num_footnotes'] = data['text'].apply(find_num_footnotes)
    data['num_note_tags'] = data['text'].apply(find_num_note_tags)
    data['num_underlines'] = data['text'].apply(find_num_underlines)
    data['num_journal_citations'] = data['text'].apply(find_num_journal_citations)
    data['num_about_links'] = data['text'].apply(find_num_about_links)
    data['num_wikitables'] = data['text'].apply(find_num_wikitables)
    data.dropna(inplace=True)
    X = data.loc[:, ['has_infobox','num_categories','num_images','num_ISBN','num_references','article_length',
                'num_difficult_words','dale_chall_readability_score','readability_index','linsear_write_formula',
                'gunning_fog_index', 'num_web_citations','num_book_citations','num_news_citations',
                'num_quotes','num_h3_headers','num_internal_links', 'num_h2_headers', 'syllable_count',
                'lexicon_count', 'sentence_count','num_footnotes', 'num_note_tags', 'num_underlines', 'num_journal_citations',
                'num_about_links', 'num_wikitables', 'smog_index']].values
    predictions = rf_model.predict(X)
    data['Predicted_Quality'] = predictions
    Cat_page_df = data.loc[:, ['category', 'page', 'Predicted_Quality']]
    return Cat_page_df.groupby(by='category').mean()

In [18]:
%pdb

Automatic pdb calling has been turned OFF


In [ ]:
import time
get_wiki_category_quality('machine_learning')

In [ ]:
ml_data = pickle.load( open( "Machine_Learning_Category_Wikipedia.pkl", "rb" ) )

In [54]:
Cat_groups

,Predicted_Quality,category
0,1.885312,Applied machine learning
1,1.528246,Artificial intelligence conferences
2,1.668835,Artificial neural networks
3,1.622311,Bayesian networks
4,1.957163,Classification algorithms
5,1.925219,Cluster analysis
6,1.587266,Computational learning theory
7,1.669472,Data mining and machine learning software
8,1.793419,Datasets in machine learning
9,1.962699,Dimension reduction


In [ ]:
ml_data['category'] = ml_data.index

In [53]:
Cat_groups.index = list(range(len(Cat_groups)))

In [61]:
Cat_groups.to_pickle('ML_data.pkl')

In [ ]:
for i, d in ml_data.iterrows():
    print(d['category'])

In [75]:
ml_pages = pickle.load(open('ML_pages.pkl', 'rb'))

In [59]:
Cat_groups.Predicted_Quality = Cat_groups.Predicted_Quality.round(2)

In [74]:
for i, j in Cat_groups.iterrows():
    print(j)
    break

Predicted_Quality                        1.89
category             Applied machine learning
Name: 0, dtype: object


In [76]:
ml_pages

,category,page,Predicted_Quality
0,Applied machine learning,Activity recognition,2.59
1,Applied machine learning,AlchemyAPI,1.59
2,Applied machine learning,AlphaGo,3.70
3,Applied machine learning,AlphaGo Zero,2.46
4,Applied machine learning,AlphaZero,2.30
5,Applied machine learning,BigDL,0.20
6,Applied machine learning,Caffe (software),1.54
7,Applied machine learning,Chainer,2.18
8,Applied machine learning,Comparison of deep learning software,3.40
9,Applied machine learning,Cortica,2.29
